In [ ]:
from dendron_tutorial_1 import *

from whisper_mic import WhisperMic

In [ ]:
class GetVoiceInput(dendron.ActionNode):
    def __init__(self, latest_human_input_key = "latest_human_input"):
        super().__init__("get_voice_input")
        self.latest_human_input_key = latest_human_input_key
        self.mic = WhisperMic()

    def tick(self):
        t = np.arange(8000) / 16000
        t = t.reshape(-1, 1)
        beep = 0.2 * np.sin(2 * np.pi * 440 * t)
        sd.play(beep, 16000)

        self.blackboard[self.latest_human_input_key] = self.mic.listen()

        chat = self.blackboard["chat_history"]
        chat.append({"role": "GPT4 Correct User", "content" : self.blackboard[self.latest_human_input_key]})
        self.blackboard["in"] = chat

        return NodeStatus.SUCCESS

In [ ]:
speech_node = TTSAction("speech_node")
speech_node.add_post_tick(play_speech)

chat_behavior_cfg = CausalLMActionConfig(load_in_4bit=True,
                                         max_new_tokens=128,
                                         do_sample=True,
                                         top_p=0.95,
                                         use_flash_attn_2=True,
                                         model_name='openchat/openchat_3.5')

chat_node = CausalLMAction('chat_node', chat_behavior_cfg)

chat_node.set_input_processor(chat_to_str)
chat_node.set_output_processor(str_to_chat)
chat_node.add_post_tick(set_next_speech)

farewell_classifier_cfg = CompletionConditionConfig(
    input_key = "farewell_test_in",
    load_in_4bit=True,
    model_name='mlabonne/Monarch-7B',
    use_flash_attn_2=True
)

farewell_classification_node = CompletionCondition("farewell_classifier", farewell_classifier_cfg)
farewell_classification_node.add_pre_tick(farewell_pretick)
farewell_classification_node.add_post_tick(farewell_posttick)

In [ ]:
speech_seq = Sequence("speech_seq", [
    MoreToSay(),
    speech_node
])

thought_seq = Sequence("thought_seq", [
    TimeToThink(),
    chat_node,
    SentenceSplitter()
])

goodbye_test = Sequence("goodbye_test", [
    farewell_classification_node,
    SayGoodbye(), 
    speech_node
])

conversation_turn = Fallback("conversation_turn", [
    speech_seq,
    thought_seq,
    GetVoiceInput()
])

root_node = Fallback("converse", [
    goodbye_test,
    conversation_turn,
])

tree = dendron.BehaviorTree("chat_tree", root_node)

In [ ]:
tree.blackboard["chat_history"] = []
tree.blackboard["speech_in"] = []

tree.blackboard.register_entry(dendron.blackboard.BlackboardEntryMetadata(
    key = "latest_human_input",
    description = "The last thing the human said.",
    type_constructor = str
))
tree.blackboard["latest_human_input"] = None

tree.blackboard["completions_in"] = ["yes", "no"]
tree.blackboard["success_fn"] = farewell_success_fn
tree.blackboard["all_done"] = False

In [ ]:
while not tree.blackboard["all_done"]:
    tree.tick_once()